# Setup

In [ ]:
# os.environ['LC_ALL'] = 'C.UTF-8'
# os.environ['LANG'] = 'C.UTF-8'

In [ ]:
!pip install pydub

Get MusicGen git repo

In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft

Access to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Imports

In [ ]:
import os
import random
from pydub import AudioSegment

import soundfile as sf
import numpy as np

# Data Processing

Create auxiliary folders if necessary

In [ ]:
new_folders = ["raw", "output", "musicgen_trainer_dir"]

for folder_name in new_folders:
  if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"'{folder_name}'folder successfully created!")

Create necessary folders in Drive

In [ ]:
new_folders = ["raw", "output", "results"]

for folder_name in new_folders:
  file_path = os.path.join("/content/drive/MyDrive/ap-project/", folder_name)
  if not os.path.exists(file_path):
    os.makedirs(file_path)
    print(f"'{folder_name}'folder successfully created!")

In [ ]:
raw = "/content/drive/MyDrive/ap-project/raw/desgarrada"
if not os.path.exists(raw):
   os.makedirs(raw)
   print(f"'{raw}'folder successfully created!")

output = "/content/drive/MyDrive/ap-project/output/desgarrada"
if not os.path.exists(output):
   os.makedirs(output)
   print(f"'{output}'folder successfully created!")

Function to clean a directory

In [ ]:
def clean_dir(dir_path):
  for file_name in os.listdir(dir_path):
    file_path = os.path.join(dir_path, file_name)
    if os.path.isfile(file_path):
      os.remove(file_path)
      print(f"File {file_path} was deleted")

Clean output directory

In [ ]:
clean_dir('/content/drive/MyDrive/ap-project/output')

In case raw and output are not in Drive

In [ ]:
!cp -r /content/raw/* "/content/drive/MyDrive/ap-project/raw"

In [ ]:
!cp -r /content/output/* "/content/drive/MyDrive/ap-project/output"

If you upload more audios to raw's folder, clean output's folder before running process_audios function

In [ ]:
clean_dir('/content/drive/MyDrive/ap-project/output')

Function to process audio data: segments audio in 30 seconds each and save .txt labels and set sample rate to 32000 Hz

In [ ]:
def process_audios(file_path, output_dir, start_segment, segment_length=30): # 30 seconds
  # Load audio
  audio = AudioSegment.from_file(file_path)

  # Extract file name for .txt
  file_name = os.path.splitext(os.path.basename(file_path))[0]

  # Convert segment length to ms
  segment_length_ms = segment_length * 1000

  # Set the sample rate to 32000 Hz
  audio = audio.set_frame_rate(32000)

  # Calculate number of segments
  num_segments = (len(audio) + segment_length_ms - 1) // segment_length_ms

  for i in range(num_segments):
    start_time = i * segment_length_ms

    # Last segment
    if i == num_segments - 1:
      start_time = len(audio) - segment_length_ms

    end_time = start_time + segment_length_ms

    # Get segment
    segment = audio[start_time:end_time]
    print(f"i: {i} file: {file_path} - segment_{start_segment:03d}.wav")

    # Save segment
    segment.export(os.path.join(output_dir, f'segment_{start_segment:03d}.wav'), format='wav')

    # Save label
    with open(os.path.join(output_dir, f'segment_{start_segment:03d}.txt'), 'w') as f:
      f.write(file_name)
    start_segment +=1

  return start_segment

Process all audios

In [ ]:
output_dir = '/content/drive/MyDrive/ap-project/output/desgarrada'
samples_dir = '/content/drive/MyDrive/ap-project/raw/desgarrada'

if not os.path.exists(output_dir):
  os.makedirs(output_dir)

start_segment = 0

for file_name in os.listdir(samples_dir):
  print(file_name)
  if file_name.endswith('.wav') or file_name.endswith('.mp3'):
    file_path = os.path.join(samples_dir, file_name)
    current_segment = process_audios(file_path, output_dir, start_segment, segment_length=30)
    start_segment = current_segment


In [ ]:
import librosa

output_dir = "/content/drive/MyDrive/ap-project/output/desgarrada"

for file_name in os.listdir(output_dir):
  if file_name.endswith('.wav'):
    file_path = os.path.join(output_dir, file_name)
    audio, sample_rate = librosa.load(file_path, sr=None)

    if audio.shape[0] == 32000 * 30:
      print(f"{file_name} has the correct shape: {audio.shape[0]}")
    else:
      print(f"{file_name} does not have the correct shape: {audio.shape[0]}")


# Training process

Get MusicGen Trainer

In [ ]:
!cd "/content/musicgen_trainer_dir" && git clone https://github.com/chavinlo/musicgen_trainer.git

Install requirement

In [ ]:
!pip install wandb

In [ ]:
import wandb

In [ ]:
!python /content/musicgen_trainer_dir/musicgen_trainer/run.py --dataset_path /content/drive/MyDrive/ap-project/output/desgarrada --epochs 25 --batch_size 2

In [ ]:
from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
import torch

Load model

In [ ]:
model = musicgen.MusicGen.get_pretrained('small', device='cuda')
model.set_generation_params(duration=8)
model.lm.load_state_dict(torch.load('models/lm_final.pt'))

Prompt

Clean results folder if necessary

In [ ]:
clean_dir("/content/drive/MyDrive/ap-project/results")

In [ ]:
res = model.generate([
    'desgarrada'
],
  progress=True)

sample_rate = 32000

audio_array = res.cpu().numpy()
audio_array = (audio_array * 32767).astype('int16')

audio_segment = AudioSegment(
    audio_array.tobytes(),
    frame_rate=sample_rate,
    sample_width=audio_array.dtype.itemsize,
    channels=1
)

output_dir = "/content/drive/MyDrive/ap-project/results"

existing_files = [f for f in os.listdir(output_dir) if f.startswith("audio") and f.endswith(".mp3")]
existing_numbers = [int(f[5:-4]) for f in existing_files if f[5:-4].isdigit()]
next_number = max(existing_numbers, default=0) + 1

output_path = os.path.join(output_dir, f"audio{next_number}.wav")

audio_segment.export(output_path, format="wav")

print(f"Audio saved at '{output_path}'")

display_audio(res, 32000)